# 06 - Modelado Predictivo de Exceso Epidemico

**Proyecto:** EDA de Dengue en Colombia  
**Maestria en Inteligencia Artificial** - Desarrollo de Soluciones  

Este notebook entrena modelos de clasificacion para predecir si un municipio experimentara **exceso epidemico** de dengue en un mes dado, utilizando:
- Variables climaticas y sus rezagos temporales
- Indicadores epidemiologicos (proporciones, tasas)
- Datos demograficos y poblacionales

**Modelos:** Logistic Regression, Random Forest, XGBoost  
**Datos:** `data/processed/panel_municipal_mensual.parquet`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, roc_auc_score, roc_curve,
    f1_score, precision_score, recall_score, accuracy_score
)
from xgboost import XGBClassifier

sys.path.insert(0, os.path.join('..', 'src'))
from utils import configurar_estilo, PROJECT_ROOT

configurar_estilo()
pd.set_option('display.max_columns', 80)
pd.set_option('display.max_rows', 100)

print('Librerias cargadas correctamente')

## 1. Carga y preparacion de datos

In [ ]:
# Cargar panel municipal-mensual
data_path = PROJECT_ROOT / 'data' / 'processed' / 'panel_municipal_mensual.parquet'
df = pd.read_parquet(data_path)

print(f'Dimensiones: {df.shape[0]:,} filas x {df.shape[1]} columnas')
print(f'\nColumnas ({len(df.columns)}):')
for i, col in enumerate(df.columns):
    print(f'  {i+1:2d}. {col} ({df[col].dtype})', end='\n')

print(f'\nAnos disponibles: {sorted(df["ano"].unique())}')
print(f'Municipios unicos: {df["cod_mun_n_str"].nunique()}')
df.head()

## 2. Balance de la variable objetivo `exceso`

In [ ]:
# Value counts y porcentajes
print('Distribucion de la variable objetivo `exceso`:')
print(df['exceso'].value_counts())
print(f'\nPorcentajes:')
print(df['exceso'].value_counts(normalize=True).mul(100).round(2))

ratio = df['exceso'].value_counts()[0] / df['exceso'].value_counts()[1]
print(f'\nRatio clase mayoritaria / minoritaria: {ratio:.1f}:1')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Grafico de barras del balance global
counts = df['exceso'].value_counts()
bars = axes[0].bar(['Sin exceso (0)', 'Con exceso (1)'], counts.values,
                    color=['#2196F3', '#F44336'], edgecolor='white')
for bar, val in zip(bars, counts.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 200,
                 f'{val:,}\n({val/len(df)*100:.1f}%)',
                 ha='center', va='bottom', fontweight='bold')
axes[0].set_title('Balance de la variable objetivo', fontweight='bold')
axes[0].set_ylabel('Numero de observaciones')

# Balance por ano
balance_ano = df.groupby('ano')['exceso'].mean().mul(100)
bars2 = axes[1].bar(balance_ano.index.astype(str), balance_ano.values,
                     color='#F44336', edgecolor='white', alpha=0.8)
for bar, val in zip(bars2, balance_ano.values):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
                 f'{val:.1f}%', ha='center', va='bottom', fontweight='bold')
axes[1].set_title('Porcentaje de exceso epidemico por ano', fontweight='bold')
axes[1].set_ylabel('% municipios-mes con exceso')
axes[1].set_xlabel('Ano')

plt.tight_layout()
plt.show()

print('Conclusion: La variable objetivo esta muy desbalanceada (~5.3% positivo).')
print('Se usaran tecnicas de manejo de desbalance: class_weight="balanced" y scale_pos_weight.')

## 3. Seleccion de features y target

In [ ]:
# Definir features
# Clima actuales
feats_clima = ['temperatura_c', 'precipitacion_mm', 'ndvi', 'dewpoint_c']

# Lags climaticos (1-3 meses)
feats_clima_lags = [f'{v}_lag{l}' for v in feats_clima for l in [1, 2, 3]]

# Medias moviles climaticas
feats_clima_mm = [f'{v}_mm3' for v in feats_clima]

# Lags de casos y tasa
feats_epi_lags = [
    'casos_total_lag1', 'casos_total_lag2', 'casos_total_lag3',
    'tasa_incidencia_lag1', 'tasa_incidencia_lag2', 'tasa_incidencia_lag3',
]

# Medias moviles epidemiologicas
feats_epi_mm = ['casos_total_mm3', 'tasa_incidencia_mm3']

# Demograficos y poblacion
feats_demo = ['prop_grave', 'prop_hospitalizado', 'prop_femenino', 'poblacion']

# Concatenar todos los features
FEATURES = feats_clima + feats_clima_lags + feats_clima_mm + feats_epi_lags + feats_epi_mm + feats_demo
TARGET = 'exceso'

print(f'Total features seleccionados: {len(FEATURES)}')
print(f'\nFeatures climaticos actuales ({len(feats_clima)}): {feats_clima}')
print(f'Features climaticos lags ({len(feats_clima_lags)}): {feats_clima_lags}')
print(f'Features climaticos mm3 ({len(feats_clima_mm)}): {feats_clima_mm}')
print(f'Features epidemiologicos lags ({len(feats_epi_lags)}): {feats_epi_lags}')
print(f'Features epidemiologicos mm3 ({len(feats_epi_mm)}): {feats_epi_mm}')
print(f'Features demograficos ({len(feats_demo)}): {feats_demo}')
print(f'\nTarget: {TARGET}')

# Variables excluidas intencionalmente
excluidas = [
    'cod_dpto_n_str', 'cod_mun_n_str', 'Departamento_Notificacion',
    'Municipio_notificacion', 'ano', 'mes',
    'casos_total', 'casos_regular', 'casos_grave',
    'hospitalizaciones', 'fallecidos', 'edad_media',
    'n_femenino', 'n_masculino', 'tasa_incidencia',
    'media_hist', 'std_hist', 'umbral_exceso',
    'media_tasa_hist', 'std_tasa_hist', 'umbral_exceso_tasa',
    'exceso', 'exceso_tasa'
]
print(f'\nVariables excluidas ({len(excluidas)}): IDs, conteos directos, estadisticas historicas, targets')

In [ ]:
# Preparar dataset: dropear filas con NaN en features (lags de enero generan NaN)
df_model = df[FEATURES + [TARGET, 'ano']].copy()

print(f'Antes de dropear NaN: {len(df_model):,} filas')
print(f'\nNaN por feature:')
nans = df_model[FEATURES].isnull().sum()
nans_pct = (nans / len(df_model) * 100).round(1)
nan_df = pd.DataFrame({'nulos': nans, 'pct': nans_pct})
print(nan_df[nan_df['nulos'] > 0].to_string())

df_model = df_model.dropna(subset=FEATURES)
print(f'\nDespues de dropear NaN: {len(df_model):,} filas ({len(df_model)/len(df)*100:.1f}% del total)')

## 4. Split temporal train/test

In [ ]:
# Split temporal: Train = 2010, 2016, 2019, 2022 | Test = 2024
anos_train = [2010, 2016, 2019, 2022]
anos_test = [2024]

train_mask = df_model['ano'].isin(anos_train)
test_mask = df_model['ano'].isin(anos_test)

X_train = df_model.loc[train_mask, FEATURES]
y_train = df_model.loc[train_mask, TARGET]
X_test = df_model.loc[test_mask, FEATURES]
y_test = df_model.loc[test_mask, TARGET]

print(f'Split temporal:')
print(f'  Train ({anos_train}): {len(X_train):,} filas')
print(f'  Test  ({anos_test}):  {len(X_test):,} filas')
print(f'\nBalance en Train:')
print(f'  Clase 0: {(y_train == 0).sum():,} ({(y_train == 0).mean()*100:.1f}%)')
print(f'  Clase 1: {(y_train == 1).sum():,} ({(y_train == 1).mean()*100:.1f}%)')
print(f'\nBalance en Test:')
print(f'  Clase 0: {(y_test == 0).sum():,} ({(y_test == 0).mean()*100:.1f}%)')
print(f'  Clase 1: {(y_test == 1).sum():,} ({(y_test == 1).mean()*100:.1f}%)')

## 5. Modelo 1: Logistic Regression

In [ ]:
# Escalar features para Logistic Regression
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar modelo
lr = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42,
    solver='lbfgs'
)
lr.fit(X_train_scaled, y_train)

# Predicciones
y_pred_lr = lr.predict(X_test_scaled)
y_prob_lr = lr.predict_proba(X_test_scaled)[:, 1]

# Metricas
print('=== Logistic Regression ===')
print(f'\nClassification Report:')
print(classification_report(y_test, y_pred_lr, target_names=['Sin exceso', 'Con exceso']))

roc_auc_lr = roc_auc_score(y_test, y_prob_lr)
print(f'ROC-AUC: {roc_auc_lr:.4f}')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion Matrix
cm_lr = confusion_matrix(y_test, y_pred_lr)
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Sin exceso', 'Con exceso'],
            yticklabels=['Sin exceso', 'Con exceso'])
axes[0].set_title('Matriz de Confusion - Logistic Regression', fontweight='bold')
axes[0].set_ylabel('Real')
axes[0].set_xlabel('Predicho')

# Curva ROC
fpr_lr, tpr_lr, _ = roc_curve(y_test, y_prob_lr)
axes[1].plot(fpr_lr, tpr_lr, color='#2196F3', lw=2,
             label=f'Logistic Regression (AUC = {roc_auc_lr:.4f})')
axes[1].plot([0, 1], [0, 1], 'k--', lw=1, alpha=0.5)
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('Curva ROC - Logistic Regression', fontweight='bold')
axes[1].legend(loc='lower right')

plt.tight_layout()
plt.show()

## 6. Modelo 2: Random Forest

In [ ]:
# Entrenar modelo (no requiere escalado)
rf = RandomForestClassifier(
    n_estimators=200,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

# Predicciones
y_pred_rf = rf.predict(X_test)
y_prob_rf = rf.predict_proba(X_test)[:, 1]

# Metricas
print('=== Random Forest ===')
print(f'\nClassification Report:')
print(classification_report(y_test, y_pred_rf, target_names=['Sin exceso', 'Con exceso']))

roc_auc_rf = roc_auc_score(y_test, y_prob_rf)
print(f'ROC-AUC: {roc_auc_rf:.4f}')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

# Confusion Matrix
cm_rf = confusion_matrix(y_test, y_pred_rf)
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Greens', ax=axes[0],
            xticklabels=['Sin exceso', 'Con exceso'],
            yticklabels=['Sin exceso', 'Con exceso'])
axes[0].set_title('Matriz de Confusion - Random Forest', fontweight='bold')
axes[0].set_ylabel('Real')
axes[0].set_xlabel('Predicho')

# Curva ROC
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_prob_rf)
axes[1].plot(fpr_rf, tpr_rf, color='#4CAF50', lw=2,
             label=f'Random Forest (AUC = {roc_auc_rf:.4f})')
axes[1].plot([0, 1], [0, 1], 'k--', lw=1, alpha=0.5)
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('Curva ROC - Random Forest', fontweight='bold')
axes[1].legend(loc='lower right')

# Feature Importance Top 15
importances = pd.Series(rf.feature_importances_, index=FEATURES)
top15_rf = importances.nlargest(15)
top15_rf.sort_values().plot(kind='barh', ax=axes[2], color='#4CAF50', edgecolor='white')
axes[2].set_title('Top 15 Features - Random Forest', fontweight='bold')
axes[2].set_xlabel('Importancia')

plt.tight_layout()
plt.show()

## 7. Modelo 3: XGBoost

In [ ]:
# Calcular scale_pos_weight para desbalance
n_neg = (y_train == 0).sum()
n_pos = (y_train == 1).sum()
spw = n_neg / n_pos
print(f'scale_pos_weight = {n_neg}/{n_pos} = {spw:.2f}')

# Entrenar modelo
xgb = XGBClassifier(
    n_estimators=200,
    max_depth=6,
    learning_rate=0.1,
    scale_pos_weight=spw,
    random_state=42,
    eval_metric='logloss',
    n_jobs=-1
)
xgb.fit(X_train, y_train)

# Predicciones
y_pred_xgb = xgb.predict(X_test)
y_prob_xgb = xgb.predict_proba(X_test)[:, 1]

# Metricas
print('\n=== XGBoost ===')
print(f'\nClassification Report:')
print(classification_report(y_test, y_pred_xgb, target_names=['Sin exceso', 'Con exceso']))

roc_auc_xgb = roc_auc_score(y_test, y_prob_xgb)
print(f'ROC-AUC: {roc_auc_xgb:.4f}')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(20, 5))

# Confusion Matrix
cm_xgb = confusion_matrix(y_test, y_pred_xgb)
sns.heatmap(cm_xgb, annot=True, fmt='d', cmap='Oranges', ax=axes[0],
            xticklabels=['Sin exceso', 'Con exceso'],
            yticklabels=['Sin exceso', 'Con exceso'])
axes[0].set_title('Matriz de Confusion - XGBoost', fontweight='bold')
axes[0].set_ylabel('Real')
axes[0].set_xlabel('Predicho')

# Curva ROC
fpr_xgb, tpr_xgb, _ = roc_curve(y_test, y_prob_xgb)
axes[1].plot(fpr_xgb, tpr_xgb, color='#FF9800', lw=2,
             label=f'XGBoost (AUC = {roc_auc_xgb:.4f})')
axes[1].plot([0, 1], [0, 1], 'k--', lw=1, alpha=0.5)
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('Curva ROC - XGBoost', fontweight='bold')
axes[1].legend(loc='lower right')

# Feature Importance Top 15
importances_xgb = pd.Series(xgb.feature_importances_, index=FEATURES)
top15_xgb = importances_xgb.nlargest(15)
top15_xgb.sort_values().plot(kind='barh', ax=axes[2], color='#FF9800', edgecolor='white')
axes[2].set_title('Top 15 Features - XGBoost', fontweight='bold')
axes[2].set_xlabel('Importancia')

plt.tight_layout()
plt.show()

## 8. Comparacion de modelos

In [ ]:
# Tabla resumen de metricas
modelos = {
    'Logistic Regression': (y_pred_lr, y_prob_lr),
    'Random Forest': (y_pred_rf, y_prob_rf),
    'XGBoost': (y_pred_xgb, y_prob_xgb),
}

resultados = []
for nombre, (y_pred, y_prob) in modelos.items():
    resultados.append({
        'Modelo': nombre,
        'Accuracy': accuracy_score(y_test, y_pred),
        'Precision (exceso)': precision_score(y_test, y_pred),
        'Recall (exceso)': recall_score(y_test, y_pred),
        'F1-Score (exceso)': f1_score(y_test, y_pred),
        'ROC-AUC': roc_auc_score(y_test, y_prob),
    })

df_resultados = pd.DataFrame(resultados).set_index('Modelo')
print('=== Comparacion de Modelos (Test 2024) ===')
print(df_resultados.round(4).to_string())

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Grafico comparativo de metricas
metricas_plot = df_resultados[['F1-Score (exceso)', 'ROC-AUC', 'Recall (exceso)', 'Precision (exceso)']]
metricas_plot.plot(kind='bar', ax=axes[0], edgecolor='white', width=0.7)
axes[0].set_title('Comparacion de Metricas por Modelo', fontweight='bold')
axes[0].set_ylabel('Score')
axes[0].set_ylim(0, 1)
axes[0].legend(loc='upper left', fontsize=9)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=0)

# Curvas ROC comparativas
axes[1].plot(fpr_lr, tpr_lr, color='#2196F3', lw=2,
             label=f'Logistic Regression (AUC={roc_auc_lr:.4f})')
axes[1].plot(fpr_rf, tpr_rf, color='#4CAF50', lw=2,
             label=f'Random Forest (AUC={roc_auc_rf:.4f})')
axes[1].plot(fpr_xgb, tpr_xgb, color='#FF9800', lw=2,
             label=f'XGBoost (AUC={roc_auc_xgb:.4f})')
axes[1].plot([0, 1], [0, 1], 'k--', lw=1, alpha=0.5)
axes[1].set_xlabel('False Positive Rate')
axes[1].set_ylabel('True Positive Rate')
axes[1].set_title('Curvas ROC Comparativas', fontweight='bold')
axes[1].legend(loc='lower right')

plt.tight_layout()
plt.show()

In [ ]:
# Conclusiones
mejor_f1 = df_resultados['F1-Score (exceso)'].idxmax()
mejor_auc = df_resultados['ROC-AUC'].idxmax()

print('=' * 60)
print('CONCLUSIONES')
print('=' * 60)
print(f'\n1. Mejor modelo por F1-Score: {mejor_f1} ({df_resultados.loc[mejor_f1, "F1-Score (exceso)"]:.4f})')
print(f'2. Mejor modelo por ROC-AUC:  {mejor_auc} ({df_resultados.loc[mejor_auc, "ROC-AUC"]:.4f})')
print(f'\n3. El dataset esta altamente desbalanceado (~5% positivo),')
print(f'   por lo que las tecnicas de class_weight/scale_pos_weight')
print(f'   son fundamentales para obtener recall aceptable en la clase minoritaria.')
print(f'\n4. Las metricas principales son F1-Score y ROC-AUC,')
print(f'   ya que accuracy no es informativa con clases tan desbalanceadas.')
print(f'\n5. Resumen de metricas finales:')
print(df_resultados[['F1-Score (exceso)', 'ROC-AUC']].round(4).to_string())